In [13]:
import numpy as np
import pandas as pd
from sklearn import model_selection, metrics
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import LeaveOneOut
from copy import copy 
from sklearn import svm
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.decomposition import PCA

#### Data Preprocessing

In [ ]:
df = pd.read_csv("data/Cancer_data.csv")

# print(df.info())

missing_values = df.isna().sum()
# print(f"Missing values:\n{missing_values[missing_values > 0]}")  # Only prints columns with missing values

# Transform feature from string to binary. Benign is now '0', and Malignant is '1'.
df["diagnosis"] = df["diagnosis"].map({"B": 0, "M": 1})

# print(df.iloc[0])

# Select target attribute
y = df["diagnosis"]

# Drop useless features
X = df.drop(["Unnamed: 32", "diagnosis", "id"], axis=1)
# print(df.columns)
X = np.array(X)
y = np.array(y)


scaler = StandardScaler()
X = scaler.fit_transform(X)

# Manual standardization
# mean = np.mean(X, axis=0)  # Compute mean for each feature
# std = np.std(X, axis=0)  # Compute standard deviation for each feature
# # Avoid division by zero
# std[std == 0] = 1  
# X = (X - mean) / std  # Standardization formula

# NOTE: Might be worth handling class imbalances somehow, currently it is 357(benign) to 212(malignant)
# NOTE: Consider other model performance measurements than simply missclassification rate, maybe specificity(FNR). Consider ROC/AUC
# NOTE: Set random states to make results reproducible
# NOTE: Show confusion matrix?
# NOTE: Is my kfold_evaluation a sensible way to evaluate bagged and boosted models?
# NOTE: Include explainability of final model?
pd.DataFrame(X).describe()



,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02
mean,-1.373633e-16,6.868164e-17,-1.248757e-16,-2.185325e-16,-8.366672e-16,1.873136e-16,4.995028e-17,-4.995028e-17,1.748260e-16,4.745277e-16,...,-8.241796e-16,1.248757e-17,-3.746271e-16,0.000000,-2.372638e-16,-3.371644e-16,7.492542e-17,2.247763e-16,2.622390e-16,-5.744282e-16
std,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,...,1.000880e+00,1.000880e+00,1.000880e+00,1.000880,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00,1.000880e+00
min,-2.029648e+00,-2.229249e+00,-1.984504e+00,-1.454443e+00,-3.112085e+00,-1.610136e+00,-1.114873e+00,-1.261820e+00,-2.744117e+00,-1.819865e+00,...,-1.726901e+00,-2.223994e+00,-1.693361e+00,-1.222423,-2.682695e+00,-1.443878e+00,-1.305831e+00,-1.745063e+00,-2.160960e+00,-1.601839e+00
25%,-6.893853e-01,-7.259631e-01,-6.919555e-01,-6.671955e-01,-7.109628e-01,-7.470860e-01,-7.437479e-01,-7.379438e-01,-7.032397e-01,-7.226392e-01,...,-6.749213e-01,-7.486293e-01,-6.895783e-01,-0.642136,-6.912304e-01,-6.810833e-01,-7.565142e-01,-7.563999e-01,-6.418637e-01,-6.919118e-01
50%,-2.150816e-01,-1.046362e-01,-2.359800e-01,-2.951869e-01,-3.489108e-02,-2.219405e-01,-3.422399e-01,-3.977212e-01,-7.162650e-02,-1.782793e-01,...,-2.690395e-01,-4.351564e-02,-2.859802e-01,-0.341181,-4.684277e-02,-2.695009e-01,-2.182321e-01,-2.234689e-01,-1.274095e-01,-2.164441e-01
75%,4.693926e-01,5.841756e-01,4.996769e-01,3.635073e-01,6.361990e-01,4.938569e-01,5.260619e-01,6.469351e-01,5.307792e-01,4.709834e-01,...,5.220158e-01,6.583411e-01,5.402790e-01,0.357589,5.975448e-01,5.396688e-01,5.311411e-01,7.125100e-01,4.501382e-01,4.507624e-01
max,3.971288e+00,4.651889e+00,3.976130e+00,5.250529e+00,4.770911e+00,4.568425e+00,4.243589e+00,3.927930e+00,4.484751e+00,4.910919e+00,...,4.094189e+00,3.885905e+00,4.287337e+00,5.930172,3.955374e+00,5.112877e+00,4.700669e+00,2.685877e+00,6.046041e+00,6.846856e+00


In [15]:
# pca_full = PCA().fit(X)
# plt.figure(figsize=(8, 5))
# plt.plot(np.cumsum(pca_full.explained_variance_ratio_), marker='o')
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.grid(True)
# plt.title('PCA - Explained Variance vs. Number of Components')
# plt.show()  

In [16]:
pca = PCA(n_components=0.95).fit(X)
X_pca = pca.transform(X)
# X_pca = pca.fit_transform(X)
comp = pca.components_
print(comp.shape)
print(X_pca.shape)
# print(X_pca.shape)

# Check how many components were selected
# print(f"Number of components selected: {X_pca.shape[1]}")

(10, 30)
(569, 10)


#### Building Base Models

In [17]:
class BaseLogisticRegression:
    def __init__(self, learning_rate=0.00001, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations=num_iterations
        self.weights = None

    def sigmoid(self, x):
        return 1 / (1+np.exp(-x))
        # return np.where(x >= 0, 
        #             1 / (1 + np.exp(-x)), 
        #             np.exp(x) / (1 + np.exp(x)))

    def fit(self, X, y):
        X = np.hstack([np.ones([X.shape[0], 1]), X])

        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        for i in range(self.num_iterations):
            p = self.sigmoid(X @ self.weights)
            lossGradient = X.T @ (y - p) / num_samples
            self.weights = self.weights + self.learning_rate * (lossGradient)
            
            # Print loss
            # if i % 100 == 0:  # Print every 100 iterations
            #     loss = -np.mean(y * np.log(p + 1e-15) + (1 - y) * np.log(1 - p + 1e-15))
            #     print(f"Iteration {i}: Loss = {loss:.4f}")

    def predict_proba(self, X):
        X = np.hstack([np.ones([X.shape[0], 1]), X])
        probs = self.sigmoid(X @ self.weights)
        return np.column_stack([1 - probs, probs])  # Convert to (n_samples, 2) format

    def predict(self, X, threshold=0.49864656):
        # X = np.hstack([np.ones([X.shape[0], 1]), X])
        probabilities = self.predict_proba(X)[:,1]

        # linear_combination = X @ self.weights
        # activated = self.sigmoid(linear_combination)
        # prediction = activated > threshold
        return probabilities > threshold
    
class BaseKNN:
    def __init__(self, X, y, K=5, distanceMeasure="Euclidean"):
        self.K = K
        self.X = X
        self.y = y
        self.distanceMeasure = distanceMeasure

    def fit(self, X, y):
        self.X = X
        self.y = y

    def getDistance(self, x1, x2):
        if self.distanceMeasure == "Euclidean":
            # distance = np.sqrt(np.sum((x1 - x2)**2))
            distance = np.linalg.norm(x1 - x2)
        return distance

    def get_K_NN(self, x):
        distances = np.array([self.getDistance(row, x) for row in self.X])
        K_neighbours_indices = np.argsort(distances)[:self.K]
        NN_classes = self.y[K_neighbours_indices]
        
        return NN_classes
    
    def predict(self, X):
        toPrint = [self.get_K_NN(x) for x in X]
        toPrint = np.array(toPrint)
        print(toPrint.shape)
        predictions = [stats.mode(self.get_K_NN(x), keepdims=False).mode for x in X]
        return np.array(predictions)
        # K_NN = self.get_K_NN(x)
        # # print(K_NN.shape)
        # return stats.mode(K_NN)

class BaseSoftKNN:
    def __init__(self, X, y, K=5, distanceMeasure="Euclidean"):
        self.K = K
        self.X = X
        self.y = y
        self.distanceMeasure = distanceMeasure

    def fit(self, X, y):
        self.X = X
        self.y = y

    def getDistance(self, x1, x2):
        if self.distanceMeasure == "Euclidean":
            # distance = np.sqrt(np.sum((x1 - x2)**2))
            distance = np.linalg.norm(x1 - x2)
        return distance

    def get_K_NN(self, x):
        distances = np.array([self.getDistance(row, x) for row in self.X])
        K_neighbours_indices = np.argsort(distances)[:self.K]
        NN_classes = self.y[K_neighbours_indices]
        
        return NN_classes
    
    def predict_proba(self, X):
        classes = [self.get_K_NN(x) for x in X]
        probas = np.sum(np.array(classes), axis=1) / self.K
        return np.column_stack([1 - probas, probas]) # Convert to (n_samples, 2) format
    
    def predict(self, X, threshold=0.5):
        probas = self.predict_proba(X)
        predictions = (probas > threshold).astype(int)
        return predictions



#### Evaluating Models

In [25]:
def kfold_evaluation(K, model, X, y, threshold=0.49864656):
    CV = model_selection.KFold(n_splits=K, shuffle=True)

    all_y_true = []
    all_y_prob = []

    for train_index, test_index in CV.split(X):
        X_train, y_train = X[train_index, :], y[train_index]
        X_test, y_test = X[test_index, :], y[test_index]
        
        model.fit(X_train, y_train)

        y_prob = model.predict_proba(X_test)[:, 1]
        
        all_y_true.extend(y_test)
        all_y_prob.extend(y_prob)

    y_est_global = (np.array(all_y_prob) >= threshold).astype(int)
    cm = confusion_matrix(all_y_true, y_est_global, labels=[0, 1])

    fpr_roc, tpr_roc, thresholds = metrics.roc_curve(all_y_true, all_y_prob)
    auc = metrics.auc(fpr_roc, tpr_roc)

    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0

    precision = tp / (tp + fp)
    error = (fp + fn) / len(all_y_true)
    accuracy = 1 - error
    f1_score = (2 * tp) / (2 * tp + fp + fn)
    
    return accuracy, error, f1_score, auc, fpr, tpr, fnr, tnr, precision

def kfold_evaluation_outliers_removed(K, model, X, y, threshold=0.49864656):
    CV = model_selection.KFold(n_splits=K, shuffle=True)

    all_y_true = []
    all_y_prob = []

    for train_index, test_index in CV.split(X):
        X_train, y_train = X[train_index, :], y[train_index]
        X_test, y_test = X[test_index, :], y[test_index]
        
        # mean = np.mean(X_train, axis=0)
        # std = np.std(X_train, axis=0)

        non_outliers = np.all(np.abs(X_train) < 5, axis=1)

        X_train_clean = X_train[non_outliers]
        y_train_clean = y_train[non_outliers]

        model.fit(X_train_clean, y_train_clean)

        y_prob = model.predict_proba(X_test)[:, 1]
        
        all_y_true.extend(y_test)
        all_y_prob.extend(y_prob)

    y_est_global = (np.array(all_y_prob) >= threshold).astype(int)
    cm = confusion_matrix(all_y_true, y_est_global, labels=[0, 1])

    fpr_roc, tpr_roc, thresholds = metrics.roc_curve(all_y_true, all_y_prob)
    auc = metrics.auc(fpr_roc, tpr_roc)

    tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    tnr = tn / (tn + fp) if (tn + fp) > 0 else 0

        
    error = (fp + fn) / len(all_y_true)
    accuracy = 1 - error
    f1_score = (2 * tp) / (2 * tp + fp + fn)
    
    return accuracy, error, f1_score, auc, fpr, tpr, fnr, tnr

def kfold_accuracy(K, model, X, y, threshold=0.49864656):
    CV = model_selection.KFold(n_splits=K, shuffle=True)

    errors = np.zeros(K)
    i = 0
    for train_index, test_index in CV.split(X):
        X_train, y_train = X[train_index, :], y[train_index]
        X_test, y_test = X[test_index, :], y[test_index]
        
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        fold_error = np.sum(y_pred != y_test) / len(y_test)
        errors[i] = fold_error
        i += 1

        # y_prob = model.predict_proba(X_test)[:, 1]
        
    
    
    # y_est_global = (np.array(all_y_prob) >= threshold).astype(int)
    avg_error = np.mean(errors)
        
    # error = (fp + fn) / len(all_y_true)
    accuracy = 1 - avg_error
    
    return accuracy, avg_error

def get_kfold_roc(K, model, X, y):
    CV = model_selection.KFold(n_splits=K, shuffle=True)
    
    all_y_true = []
    all_y_prob = []

    for train_index, test_index in CV.split(X):
        X_train, y_train = X[train_index, :], y[train_index]
        X_test, y_test = X[test_index, :], y[test_index]

        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)[:, 1]  # Get probability estimates

        all_y_true.extend(y_test)
        all_y_prob.extend(y_prob)

    # Compute overall ROC
    fpr, tpr, thresholds = metrics.roc_curve(all_y_true, all_y_prob)
    auc = metrics.auc(fpr, tpr)

    return fpr, tpr, auc, thresholds

def plot_roc_curve(fpr, tpr, auc):
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {auc:.2f})')
    plt.plot([0, 1], [0, 1], color='grey', linestyle='--')  # Random guess line
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


### ROC Graph and AUC Analysis 

In [26]:
# K = X.shape[0]
K = 10
fpr, tpr, auc, thresholds = get_kfold_roc(K, LogisticRegression(), X, y)
# fpr2, tpr2, auc2, thresholds2 = get_kfold_roc(10, LogisticRegression(), X, y)

# plot_roc_curve(fpr, tpr, auc)

# for i in range(len(thresholds)):
#     print(f"Threshold: {thresholds[i]:.8f}, FPR: {fpr[i]:.8f}, TPR: {tpr[i]:.8f}")

### Logistic Regression

In [27]:
import warnings 
warnings.filterwarnings("ignore")

# Compare base Logistic Regression to task Logistic Regression through K-fold cross validation
K = X.shape[0]
# K = 10
base_accuracy, base_error, base_f1_score, base_auc, base_fpr, base_tpr, base_fnr, base_tnr, base_precision = kfold_evaluation(K, BaseLogisticRegression(num_iterations=1000), X, y, threshold=0.5)
print(f"Base model accuracy: {base_accuracy}\nBase model error: {base_error}\nBase f1 Score: {base_f1_score}\nBase model AUC: {base_auc}\nBase model fnr: {base_fnr}\nBase model fpr: {base_fpr}\n")
# nonbase_accuracy, nonbase_error, nonbase_f1_score, nonbase_auc, nonbase_fpr, nonbase_tpr, nonbase_fnr, nonbase_tnr = kfold_evaluation(K, LogisticRegression(max_iter=1000), X, y, threshold=0.49864656)
nonbase_accuracy, nonbase_error, nonbase_f1_score, nonbase_auc, nonbase_fpr, nonbase_tpr, nonbase_fnr, nonbase_tnr, nonbase_precision = kfold_evaluation(K, LogisticRegression(C=0.55, max_iter=1000), X, y, threshold=0.49864656)
print(f"Nonbase model accuracy: {nonbase_accuracy}\nNonbase model error: {nonbase_error}\nNonBase f1 Score: {nonbase_f1_score}\nNonBase model AUC: {nonbase_auc}\nNonBase model fnr: {nonbase_fnr}\nNonBase model fpr: {nonbase_fpr}\nNonBase Recall: {nonbase_tpr}\nNonbase Precision: {nonbase_precision}")


Base model accuracy: 0.9332161687170475
Base model error: 0.06678383128295255
Base f1 Score: 0.910377358490566
Base model AUC: 0.9823608688758522
Base model fnr: 0.08962264150943396
Base model fpr: 0.05322128851540616

Nonbase model accuracy: 0.984182776801406
Nonbase model error: 0.015817223198594025
NonBase f1 Score: 0.9785202863961814
NonBase model AUC: 0.9950848263833835
NonBase model fnr: 0.0330188679245283
NonBase model fpr: 0.0056022408963585435
NonBase Recall: 0.9669811320754716
Nonbase Precision: 0.9903381642512077


In [ ]:
# 0.9314
# 0.9824

In [8]:
K = X.shape[0]
# K = 10
# Compare base KNN to task KNN through K-fold cross validation
# base_accuracy, base_error, base_f1_score, base_auc, base_fpr, base_tpr, base_fnr, base_tnr = kfold_evaluation(K, BaseSoftKNN(X, y, 5), X, y)
nonbase_accuracy, nonbase_error, nonbase_f1_score, nonbase_auc, nonbase_fpr, nonbase_tpr, nonbase_fnr, nonbase_tnr = kfold_evaluation_outliers_removed(K, KNeighborsClassifier(n_neighbors=10), X, y)

# print(f"Base model accuracy: {base_accuracy}\nBase model error: {base_error}\nBase f1 Score: {base_f1_score}\nBase model AUC: {base_auc}\nBase model fnr: {base_fnr}\nBase model fpr: {base_fpr}\n")
print(f"Nonbase model accuracy: {nonbase_accuracy}\nNonbase model error: {nonbase_error}\nNonBase f1 Score: {nonbase_f1_score}\nNonBase model AUC: {nonbase_auc}\nNonBase model fnr: {nonbase_fnr}\nNonBase model fpr: {nonbase_fpr}\n")

Nonbase model accuracy: 0.9718804920913884
Nonbase model error: 0.028119507908611598
NonBase f1 Score: 0.9615384615384616
NonBase model AUC: 0.9893702764124517
NonBase model fnr: 0.05660377358490566
NonBase model fpr: 0.011204481792717087



In [9]:
K = X.shape[0]
# accuracy, error = kfold_accuracy(K, svm.SVC(kernel="rbf"), X, y)
accuracy, error, f1_score, auc, fpr, tpr, fnr, tnr = kfold_evaluation_outliers_removed(K, svm.SVC(kernel="rbf", probability=True), X, y)
# model.fit(X,y)

# print(accuracy, error)    
print(f"Model accuracy: {accuracy}\nModel error: {error}\nf1 Score: {f1_score}\nModel AUC: {auc}\nModel fnr: {fnr}\nModel fpr: {fpr}\n")

Model accuracy: 0.9701230228471002
Model error: 0.029876977152899824
f1 Score: 0.9601873536299765
Model AUC: 0.9951905290418055
Model fnr: 0.0330188679245283
Model fpr: 0.028011204481792718



In [22]:
K = X.shape[0]
# K = 10
# model = BaggingClassifier(LogisticRegression(max_iter=1000))
# accuracy, error = kfold_accuracy(K, BaggingClassifier(LogisticRegression()), X, y)
# accuracy, error = kfold_accuracy(K, AdaBoostClassifier(LogisticRegression()), X, y)
accuracy, error, f1_score, auc, fpr, tpr, fnr, tnr = kfold_evaluation_outliers_removed(K, RandomForestClassifier(), X, y)
print(f"Model accuracy: {accuracy}\nModel error: {error}\nf1 Score: {f1_score}\nModel AUC: {auc}\nModel fnr: {fnr}\nModel fpr: {fpr}\n")




# print(accuracy, error)

Model accuracy: 0.9630931458699473
Model error: 0.03690685413005272
f1 Score: 0.9501187648456056
Model AUC: 0.988815337455737
Model fnr: 0.05660377358490566
Model fpr: 0.025210084033613446



In [11]:
class simpleSVM:
    def __init__(self, kernel='linear', C=1.0, degree=3, gamma=None, learning_rate=0.01, n_iters=1000):
        self.C = C
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.alpha = None
        self.b = 0
        self.X_train = None
        self.y_train = None

    def kernel_function(self, x1, x2):
        if self.kernel == 'linear':
            return x1 @ x2
        elif self.kernel == 'poly':
            return ((x1 @ x2) + 1)**self.degree
        elif self.kernel == 'rbf':
            if self.gamma is None:
                self.gamma = 1 / x1.shape[0]   
            return np.exp(-self.gamma * np.linalg.norm(x1 - x2)**2)
        else: 
            raise ValueError("Unknown kernel")


    def fit(self, X, y):
        self.X_train = copy(X)
        self.y_train = copy(y)
        n_samples, _ = X.shape
        self.alpha = np.zeros(n_samples)

        for _ in range(self.n_iters):
            for i in range(n_samples):
                gradient = 1 - y[i] * sum(self.alpha[j] * y[j] * self.kernel_function(X[i], X[j]) for j in range(n_samples))
                self.alpha[i] += self.learning_rate * gradient
                self.alpha[i] = min(max(self.alpha[i], 0), self.C)

        self.b = 1 / X.shape[0] * sum(y[i] - sum(
            self.alpha[j] * y[j] * self.kernel_function(X[j], X[i])
            for j in range(n_samples)) 
            for i in range(n_samples))
        
    def predict(self, X):
        predictions = np.sign([
            np.sum(
                self.alpha * self.y_train * [self.kernel_function(x_i, x) for x_i in self.X_train] + self.b for x in X
            )
        ])
        return predictions

In [12]:
# base_accuracy, base_avg_error = kfold_accuracy(10, simpleSVM(kernel="rbf"), X, y)

# print(f"Base SVM accuracy: {base_accuracy}\nBase SVM error: {base_avg_error}")